Frage an Lars: welche Größen Vergleiche ich? Beim Bundle ist es recht leicht: Ich schicke eine Nachricht in den dtnd und schaue, was er mir als Bundlegröße in Bytes zurückgibt.

Aber wie messe ich die Größe der NNTP-Nachricht? Ist es die Nachricht, die beim POSTEN übertragen wird (also z.B. noch ohne Message-ID, etc.) oder die, die vom Server beim Abruf, z.B. über ARTICLE, gesendet wird? Hier wären die Einsparungen naürlich etwas besser.

Wirklich glänzen kann das Bundle, wenn UTF-8 und inb Emojis übertragen werden. Und dieser Usecase ist sehr wahrscheinlich.

Ist für den NNTP-Payload die Bytes-Länge des Systems relevant? Oder die Länge in chars?

In [1]:
import requests as rq
import json
import subprocess
import cbor2

In [52]:
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [53]:
def dtnsend(data: str, sender: str, receiver: str) -> str:
    cmd = ['dtnsend', '--sender', sender, '--receiver', receiver]
    piped_data = subprocess.Popen(['echo', data], stdout=subprocess.PIPE)
    return subprocess.check_output(cmd, stdin=piped_data.stdout).decode()

In [54]:
articles_paths = {"long": "articles/long_text",
                  "med": "articles/med_text", 
                  "short": "articles/short_text"}
articles = dict()
for k, path_string in articles_paths.items():
    with open(path_string, "r") as fh:
        articles[k] = fh.readlines()

In [58]:
node_id = "florentin"

email_id = "t.e.schmitt"
email_domain = "posteo.de"
email_address = f"{email_id}@{email_domain}"
newsgroup = "monntpy.eval"

article_length = "short"
body = "".join(articles[article_length])

nntp_payload = f"""From: {email_address}
Newsgroups: {newsgroup}
Date: Mon, 22 Aug 2022 13:56:09 UTC
Subject: Corrupti commodi consequuntur fugiat corporis atque eaque
Message-ID: <714491769820-0@{node_id}-mail-{email_domain}-{email_id}.dtn>

{body}
."""

# nntp_payload = "".join(article_template + articles[article_length])
# print(f"Length of NNTP payload: {len(article_text)}")
size_nntp = len(nntp_payload)

sender = f"dtn://{node_id}/mail/{email_domain}/{email_id}"
receiver = f"dtn://{newsgroup}/~news"

send_resp = dtnsend(data=body, sender=sender, receiver=receiver)
size_bp7 = int(send_resp.splitlines()[1].split(sep=":", maxsplit=1)[1].strip().split(" ")[1])

print(f"""Size comparison:

Protocol | Size (Bytes)
--------------------------------
NNTP     | {size_nntp:6}
BP7      | {size_bp7:6}
--------------------------------
Diff     | {(size_bp7 - size_nntp):6}
--------------------------------""")

Size comparison:

Protocol | Size (Bytes)
--------------------------------
NNTP     |    429
BP7      |    351
--------------------------------
Diff     |    -78
--------------------------------


In [63]:
url = "http://localhost:3000"
bid_list_resp = rq.get(f"{url}/status/bundles")
bid_list = json.loads(bid_list_resp.content)
# bundle_from_dtnd = rq.get(f"{url}/download?{bid}")
bid_list

['dtn://florentin/mail/posteo.de/t.e.schmitt-715871801040-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715871144614-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715873957869-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715871730282-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715873856992-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715873853723-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715871807300-0',
 'dtn://florentin/mail/posteo.de/t.e.schmitt-715868440828-0']

In [47]:
data = "Arrrgh!"
dst = "dtn://eval.core.monntpy/~news"
payload: dict = {
        "source": "dtn://florentin/mail/posteo.de/t.e.schmitt",
        "destination": dst,
        "data": data,
    }
bundle_ready = pack_bundle(**payload)

{'src': 'dtn://remotenode01/mail/stud.tu-darmstadt.de/thomas.schmitt2', 'dst': 'dtn://monntpy.dev/~news', 'delivery_notification': False, 'lifetime': 86400000, 'data': 'Arrrgh!'}


In [39]:
def pack_bundle(destination: str, data: str, source: str = None,):
    bundle_dict: dict = {
        "src": source,
        "dst": destination,
        "delivery_notification": False,
        "lifetime": 24 * 3600 * 1000,
        "data": data,
    }
    print(bundle_dict)

    return cbor2.dumps(bundle_dict)